In [1]:
import numpy as np
import pandas as pd
import open3d as o3d
import os
import glob

In [2]:
input_path = "../data/"
output_path = "../data/"
data_noise = "group_0.csv"
data_noise_hits = "group_1637.csv"

# Noise Only

In [3]:
noise_cloud = pd.read_csv(input_path + data_noise)

Convert dataframe to numpy arrary containing list of lists where each list contains a row of x, y, z points

In [4]:
noise_cloud = noise_cloud.values

First instantiate the Open3d point cloud object, then add points

In [5]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(noise_cloud)

Need to estimate normals for a future BPA algorithm

In [6]:
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=300))

True

In [7]:
pcd.normals = o3d.utility.Vector3dVector(np.asarray(pcd.normals))

In [8]:
pcd.points

std::vector<Eigen::Vector3d> with 12454 elements.
Use numpy.asarray() to access data.

In [ ]:
o3d.visualization.draw_geometries([pcd])

## Mesh

### Strategy 1: Ball-Pivoting Algorithm
The idea behind the Ball-Pivoting Algorithm (BPA) is to simulate the use of a virtual ball to generate a mesh from a point cloud. We first assume that the given point cloud consists of points sampled from the surface of an object. Points must strictly represent a surface (noise-free), that the reconstructed mesh explicit.Using this assumption, imagine rolling a tiny ball across the point cloud “surface”. This tiny ball is dependent on the scale of the mesh, and should be slightly larger than the average space between points. When you drop a ball onto the surface of points, the ball will get caught and settle upon three points that will form the seed triangle. From that location, the ball rolls along the triangle edge formed from two points. The ball then settles in a new location: a new triangle is formed from two of the previous vertices and one new triangle is added to the mesh. As we continue rolling and pivoting the ball, new triangles are formed and added to the mesh. The ball continues rolling and rolling until the mesh is fully formed.

The idea behind the Ball-Pivoting Algorithm is simple, but of course, there are many caveats to the procedure as originally expressed here:
1. How is the ball radius chosen? The radius, is obtained empirically based on the size and scale of the input point cloud. In theory, the diameter of the ball should be slightly larger than the average distance between points.
2. What if the points are too far apart at some locations and the ball falls through? When the ball pivots along an edge, it may miss the appropriate point on the surface and instead hit another point on the object or even exactly its three old points. In this case, we check that the normal of the new triangle Facet is consistently oriented with the point's Vertex normals. If it is not, then we reject that triangle and create a hole.
3. What if the surface has a crease or valley, such that the distance between the surface and itself is less than the size of the ball? In this case, the ball would just roll over the crease and ignore the points within the crease. But, this is not ideal behavior as the reconstructed mesh is not accurate to the object.
4. What if the surface is spaced into regions of points such that the ball cannot successfully roll between the regions? The virtual ball is dropped onto the surface multiple times at varying locations. This ensures that the ball captures the entire mesh, even when the points are inconsistently spaced out.


First compute the necessary radius parameter based on the average distances computed from all the distances between points:

In [9]:
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist

We can then create a mesh and store it in the bpa_mesh variable:

In [13]:
radii = o3d.utility.DoubleVector([radius, radius * 2])
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, radii)

TODO: DETERMINE IF NEEDED:
Before exporting the mesh, we can downsample the result to an acceptable number of triangles, for example, 100k triangles:

In [14]:
dec_mesh = bpa_mesh.simplify_quadric_decimation(100000)

### Strategy 2: Poisson Reconstruction
The Poisson Reconstruction is a bit more technical/mathematical. Its approach is known as an implicit meshing method, which I would describe as trying to “envelop” the data in a smooth cloth. Without going into too many details, we try to fit a watertight surface from the original point set by creating an entirely new point set representing an isosurface linked to the normals. There are several parameters available that affect the result of the meshing:
1. Which depth? a tree-depth is used for the reconstruction. The higher the more detailed the mesh (Default: 8). With noisy data you keep vertices in the generated mesh that are outliers but the algorithm doesn’t detect them as such. So a low value (maybe between 5 and 7) provides a smoothing effect, but you will lose detail. The higher the depth-value the higher is the resulting amount of vertices of the generated mesh.
2. Which width? This specifies the target width of the finest level of the tree structure, which is called an octree 🤯. Don’t worry, I will cover this and best data structures for 3D in another article as it extends the scope of this one. Anyway, this parameter is ignored if the depth is specified.
3. Which scale? It describes the ratio between the diameter of the cube used for reconstruction and the diameter of the samples’ bounding cube. Very abstract, the default parameter usually works well.
4. Which fit? the linear_fit parameter if set to true, let the reconstructor use linear interpolation to estimate the positions of iso-vertices.

NOTE: The function output a list composed of an o3d.geometry object followed by a Numpy array. We want to select only the o3d.geometry justifying the [0] at the end.

In [15]:
poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, 
                                                                         depth=8, 
                                                                         width=0, 
                                                                         scale=1.1, 
                                                                         linear_fit=False)[0]

TODO: Test if needed:To get a clean result, it is often necessary to add a cropping step to clean unwanted artifacts. For this, we compute the initial bounding-box containing the raw point cloud, and we use it to filter all surfaces from the mesh outside the bounding-box: